<a href="https://colab.research.google.com/github/PavlenkoDR/Neural/blob/master/classwork/Neuro%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования.

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

#!unzip -uq "/content/gdrive/My Drive/DataSet/train/train.zip" -d "/content/gdrive/My Drive/DataSet/train"

In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam

from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization

from tensorflow.python.keras.applications import InceptionV3

In [0]:
# Каталог с данными для обучения
test_dir = '/content/gdrive/My Drive/DataSet/test'
# Каталог с данными для проверки
val_dir = '/content/gdrive/My Drive/DataSet/val'
# Каталог с данными для тестирования
train_dir = '/content/gdrive/My Drive/DataSet/train/train'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 32
# Количество изображений для обучения
nb_train_samples = 2565
# Количество изображений для проверки
nb_validation_samples = 100
# Количество изображений для тестирования
nb_test_samples = 100

Загружаем предварительно обученную нейронную сеть

In [0]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Загружаем предварительно обученную нейронную сеть Inceptionv3
#inception_net = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Создаем составную нейронную сеть на основе VGG16 или inception

In [23]:
vgg16_net.summary()
#inception_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
#model.add(inception_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

Компилируем составную нейронную сеть

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

Создаем генератор изображений
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

Генератор данных для обучения на основе изображений из каталога

In [27]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 14034 images belonging to 6 classes.


Генератор данных для проверки на основе изображений из каталога

In [28]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 126 images belonging to 6 classes.


Генератор данных для тестирования на основе изображений из каталога

In [29]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 126 images belonging to 6 classes.


Обучаем модель с использованием генераторов
train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [30]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
80/80 [==============================] - 46s 570ms/step - loss: 1.2450 - acc: 0.5181 - val_loss: 0.5582 - val_acc: 0.7708
Epoch 2/10
80/80 [==============================] - 38s 476ms/step - loss: 0.6625 - acc: 0.7582 - val_loss: 0.4119 - val_acc: 0.8333
Epoch 3/10
80/80 [==============================] - 38s 477ms/step - loss: 0.4813 - acc: 0.8309 - val_loss: 0.3464 - val_acc: 0.8854
Epoch 4/10
80/80 [==============================] - 38s 477ms/step - loss: 0.4350 - acc: 0.8465 - val_loss: 0.3665 - val_acc: 0.8542
Epoch 5/10
80/80 [==============================] - 38s 477ms/step - loss: 0.3838 - acc: 0.8605 - val_loss: 0.3101 - val_acc: 0.8646
Epoch 6/10
80/80 [==============================] - 38s 476ms/step - loss: 0.3566 - acc: 0.8797 - val_loss: 0.2759 - val_acc: 0.8854
Epoch 7/10
80/80 [==============================] - 38s 476ms/step - loss: 0.3133 - acc: 0.8980 - val_loss: 0.2410 - val_acc: 0.8958
Epoch 8/10
80/80 [==============================] - 38s 476ms/step - 

In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [32]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))


Аккуратность на тестовых данных: 93.75%


In [38]:

from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.keras.applications.resnet50 import preprocess_input, decode_predictions

image_file_name = test_dir + '/sea/20212.jpg'
img = image.load_img(image_file_name, target_size=(150, 150))
#plt.imshow(img)
#plt.show()

xxxxx = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)

xxxxx = xxxxx.reshape((1,) + xxxxx.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

xxxxx = preprocess_input(xxxxx)
preds = model.predict(xxxxx)
print('Predicted:', preds)
	
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

#model.save_weights(name)
print(name + " saved model to disk")


Predicted: [[7.5445092e-22 0.0000000e+00 2.9449462e-04 9.0023826e-16 9.9970549e-01
  1.5608179e-35]]
Аккуратность на тестовых данных: 93.75%


NameError: ignored